In [1]:
from sage.all import *
from sage.schemes.elliptic_curves.hom_frobenius import EllipticCurveHom_frobenius

#HR import
from sage.schemes.elliptic_curves.hom_composite import EllipticCurveHom_composite
from time import time

from parameters.parameter_generation import read_params, find_param, find_param_gen, save_params
from utilities.supersingular import random_point, compute_point_order_D, torsion_basis
from isogenies.Kani_endomorphism import KaniEndo, KaniEndoHalf
from theta_structures.Tuple_point import TuplePoint
from montgomery_isogenies.isogenies_x_only import isogeny_from_scalar_x_only, evaluate_isogeny_x_only
from basis_change.canonical_basis_dim1 import make_canonical
from utilities.strategy import precompute_strategy_with_first_eval, precompute_strategy_with_first_eval_and_splitting

The goal is to construct a 4D representation of an endomorphism of large degree over a cryptographically large field.

In [2]:
# Construction of a curve E1 with no additional automorphisms from E1728

p = (2^409)*31 - 1;
K.<i>=GF(p**2,'i',modulus=[1,0,1],proof=False)
E1728 = EllipticCurve(K,[1,0]).montgomery_model();

# A 31-torsion basis
P31 = E1728(18164032128839059988568742675776842209139618729855461560483615855944772133627992670220534135836213794768222809093040385156630*i + 31242978218487975138283563260665570176268755555201255995607305749422321088070245703832087186673769953903044664369187512278091,12156818949863445228081066556995488984938032059032694318421574269020953231081745400636909490499799488543659937317844326740665*i + 23571854642288383361165406676793687762832597517437623283240470804449567011820052890451084989166297337333370220875734444231894,1);
Q31 = E1728(25202808321769688784710477507373840344762159744055984394958702647907595074668954588547727611194733258934536364750558628814838*i + 2013827789490608703731879760397100876683602176029348027311721566436449896204706401004434564771846492111048293166204645255549,30845114248479757312430552687614220582768522984408689306504296874146708859378389210267190147187757933792998911162327460471394*i + 32736763418012592783002500903721064162756913358764017618708499381157290081935893757119675284364339599072981985107366889822241);

# A random 31-isogeny
T = 24*P31 + 29*Q31;
phi = E1728.isogeny(T,model="montgomery");
E1 = phi.codomain();

# We compute a basis of the 2^409-torsion
P2,Q2 = E1728.torsion_basis(2^409);
P2 = phi(P2); Q2 = phi(Q2);
phi_dual = phi.dual();

In [3]:
# We almost construct a basis of the endomorphism ring (we take index 2 subring)
almost_End_E1728 = [E1728.identity_morphism()];
for j in E1728.automorphisms():
    if j^2 == -1:
        almost_End_E1728 += [j];
        break;
almost_End_E1728 += [EllipticCurveHom_frobenius(E1728), EllipticCurveHom_frobenius(E1728)*almost_End_E1728[1]]; # = [identity, iota:(x,y) |-> (x,alpha y) with alpha^2 = -1,frobenius, iota*frobenius]

In [8]:
# We take an endomorphism in this subring that can be represented using Dim 4
a,b,c,d = [11467137290107925521157662155391785363462553153077205334749603008212229901718577292,
 171402665276917795260544755764090323812980,
 49130984663729591851,
 74954008488733524786];
theta = E1728.scalar_multiplication(a) + E1728.scalar_multiplication(b)*almost_End_E1728[1] + E1728.scalar_multiplication(c)*almost_End_E1728[2] + E1728.scalar_multiplication(d)*almost_End_E1728[3];
deg_theta = a^2 + b^2 + (c^2)*p + (d^2)*p; # Coming from the fact that theta = a + b*iota + c*pi + d*iota*pi
a1,a2 = (475154821954838321606, 1938663281226055004821);
e,f = (547, 276);

In [9]:
# We check if we have the correct relation
2^e == deg_theta + a1^2 + a2^2

True

From this setup : $ E_{1728} \rightarrow^{\theta} E_{1728} \rightarrow^{\phi} E_1 \rightarrow^{\psi} E_1$
where $\psi = \frac{\phi \circ \theta \circ \hat{\phi}}{31}$, we want to compute a 4D representation of $\psi$. To this end, we use the fact that $\deg \psi = \deg \theta$, thus we have
$\deg \psi + a1^2 + a2^2 = 2^e$.

In [10]:
# We construct a f-torsion basis over E1
ff = 409 - f;
P = 2^ff*P2; Q = 2^ff*Q2; 

In [11]:
R = phi(theta(phi_dual(P))); S = phi(theta(phi_dual(Q))); # We apply the lollipop endomorphism

In [12]:
R = inverse_mod(31,2^f)*R; S = inverse_mod(31,2^f)*S; # We divide by deg(phi) to conserve the equality to a sum of two squares

In [13]:
F = KaniEndoHalf(P,Q,R,S,deg_theta,a1,a2,e,f)

In [30]:
F.F1._isogenies[-1].codomain().hadamard().null_point()==F.F2_dual._isogenies[-1].codomain().null_point()

False

In [10]:
#Let's try to apply the endomorphism

In [14]:
PP = TuplePoint([E1.random_point(), E1(0), E1(0), E1(0)]);

In [15]:
F(PP)

AssertionError: 

In [13]:
# I'm also trying with P.parent() as E1 is apparently different.
P.parent() == Q.parent() == R.parent() == S.parent() != E1

True

In [14]:
PP = TuplePoint([P.parent()(E1.random_point()), P.parent()(0),P.parent()(0),P.parent()(0)]);

In [15]:
F(PP)

AssertionError: 

Testing with other endomorphisms:

In [16]:
# Here are the functions I used to compute an endomorphism theta over E1728 together with a1, a2, e and f such that 2^e == deg_theta + a1^2 + a2^2 and f = ceil(e/2) +2

disc = 2^529
ZZbound = Integers(ceil(disc^(1/8)));
def rand_int_bound():
    return ZZ(ZZbound.random_element());
    
def find_endo():
    c,d = [rand_int_bound() for i in [1..2]];
    P = p*(c^2 + d^2);
    e = ceil(log(P,2));
    square_list = four_squares(2^e - P);
    L = [is_prime(square_list[i]^2 + P) for i in range(4)];
    while L == [0,0,0,0]:
        c,d = [rand_int_bound() for i in [1..2]];
        P = p*(c^2 + d^2);
        e = ceil(log(P,2));
        square_list = four_squares(2^e - P);
        L = [is_prime(square_list[i]^2 + P) for i in range(4)];
    
    i = L.index(1);
    b = square_list[i];
    a = square_list[mod(i+1,4)];
    a1 = square_list[mod(i+2,4)];
    a2 = square_list[mod(i+3,4)];
    
    deg_theta0 = b^2 + (c^2)*p + (d^2)*p; #in addition trace_theta0 = 0
    deg_theta = deg_theta0 + a^2;
    theta0 = (E1728.scalar_multiplication(b)*almost_End_E1728[1] + E1728.scalar_multiplication(c)*almost_End_E1728[2] + E1728.scalar_multiplication(d)*almost_End_E1728[3]);
    theta = E1728.scalar_multiplication(a) + theta0;
    tr_theta = 2*a;
    disc0 = -4*deg_theta0;
    f = ceil(e/2) + 2;
    return [a,b,c,d],a1,a2,deg_theta,theta,e,f;

In [17]:
# If you want to generate a new endomorphism
A,a1,a2,deg_theta,theta,e,f = find_endo();

In [18]:
# And test it (It does not always lead to working KaniEndoHalf):
ff = 409 - f;
P = 2^ff*P2; Q = 2^ff*Q2; 
R = phi(theta(phi_dual(P))); S = phi(theta(phi_dual(Q)));
R = inverse_mod(31,2^f)*R; S = inverse_mod(31,2^f)*S;
F = KaniEndoHalf(P,Q,R,S,deg_theta,a1,a2,e,f);

In [19]:
PP = TuplePoint([E1.random_point(), E1(0), E1(0), E1(0)]);

In [20]:
#There is always the same issue
F(PP)

AssertionError: 